<a href="https://colab.research.google.com/github/ol3gka/AI_Masrters_NLA_project_1_Matrix_Masters/blob/main/Maxvol_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# maxvol-2 (rect-maxvol)
Автор: Николаев О.В.

- добавить, что такое объем (детерминант прямоугольной матрицы)

- написать докстринг




**Algorithm 1 rect_maxvol (“Greedy” maximization of the volume of submatrix)**



<div>
<img src="https://raw.githubusercontent.com/ol3gka/AI_Masrters_NLA_project_1_Matrix_Masters/main/pictures/max_vol_1.jpg" width="600"/>
</div>

In [ ]:
def py_rect_maxvol(
    A, tol=1.,
    maxK=None,
    min_add_K=None,
    minK=None,
    start_maxvol_iters=10,
    identity_submatrix=True,
    top_k_index=-1):
    """
    Реализация алгоритма rect_maxvol:
    нахождение прямоугольной подматрицы с максимальным объемом

    Args:
      написать

    Returns:
      написать
    """
    # tol2 - квадрат параметра tol
    tol2 = tol**2
    # N - число строк, r - число колонок матрицы A
    N, r = A.shape
    # работа над параметрами
    if N <= r:
        return np.arange(N, dtype=np.int32), np.eye(N, dtype=A.dtype)
    if maxK is None or maxK > N:
        maxK = N
    if maxK < r:
        maxK = r
    if minK is None or minK < r:
        minK = r
    if minK > N:
        minK = N
    if min_add_K is not None:
        minK = max(minK, r + min_add_K)
    if minK > maxK:
        minK = maxK
        #raise ValueError('minK value cannot be greater than maxK value')
    if top_k_index == -1 or top_k_index > N:
        top_k_index = N
    if top_k_index < r:
        top_k_index = r
    # выбор изначальной подматрицы и коэффициентов согласно maxvol алгоритму
    index = np.zeros(N, dtype=np.int32)
    chosen = np.ones(top_k_index)
    tmp_index, C = py_maxvol(A, 1.05, start_maxvol_iters, top_k_index)
    index[:r] = tmp_index
    chosen[tmp_index] = 0
    C = np.asfortranarray(C)
    # вычислим квадратные 2-нормы каждой строки в матрице коэффиценттов матриы C
    row_norm_sqr = np.array([chosen[i]*np.linalg.norm(C[i], 2)**2 for
        i in range(top_k_index)])
    # находим максимальное число в row_norm_sqr
    i = np.argmax(row_norm_sqr)
    K = r
    # выберете cgeru или zgeru для комплексных чисел и dger или sger для float
    try:
        ger = get_blas_funcs('geru', [C])
    except:
        ger = get_blas_funcs('ger', [C])
    # изменяем (увеличваем) maxvol подматрицу с каждой итерацией
    while (row_norm_sqr[i] > tol2 and K < maxK) or K < minK:
        # прибавляем i к индексу и пересчитываем С и квадратные нормы для каждой
        # строки с помощью SVM-формулы
        index[K] = i
        chosen[i] = 0
        c = C[i].copy()
        v = C.dot(c.conj())
        l = 1.0/(1+v[i])
        ger(-l,v,c,a=C,overwrite_a=1)
        C = np.hstack([C, l*v.reshape(-1,1)])
        row_norm_sqr -= (l*v[:top_k_index]*v[:top_k_index].conj()).real
        row_norm_sqr *= chosen
        # найти максимальное значение в row_norm_sqr
        i = row_norm_sqr.argmax()
        K += 1
    # если identity_submatrix is True, установите подматрицу соответсвующую
    # maxvol строкам, раной identity матрице
    if identity_submatrix:
        C[index[:K]] = np.eye(K, dtype=C.dtype)
    return index[:K].copy(), C